<h3> EE 445 Spring 2023</h3>
<h3> PAC Learnability</h3>
<h4> Narayana Santhanam </h4>



This notebook is a demonstration of the PAC learning example we went over in the first week of classes. You will learn more about PAC learnability in EE645, but you should get a feel for modeling probabilistically. You should get comfortable with thinking about discrete probability with this example.

In [266]:
import numpy as np
import numpy.random as npr

In [272]:
n = 15

# A conjunction on n variables is represented as a ternary vector of length n. 
# 1 in the ith position indicates x(i) exists in the conjunction.
# 0 in the jth position indicates ~x(j) exists in the conjunction.
# 2 in the kth position indicates both x(k) and ~x(k) are absent in the conjunction.
# Numbering of variables is from 0. 
# For example if n =3 and the conjunction is x(1), we use the vector [2,1,2]. 
# If the conjunction is x(0) AND ~x(2), we use the vector [1,2,0] to represent it.
def humanreadable(c):
    # Translates the representation of the conjunction to an easily readable form.
    if len(c)==0:
        return '0'
    literals = [x for x in range(n) if c[x] !=2 ]
    conj = ''
    for i in literals:
        if c[i] == 1:
            conj+=' x('+str(i)+') AND'
        else:
            conj+=' ~x('+str(i)+') AND'
    if conj == '':
        return '1'
    else:
        return conj[:-len(' AND')]

def concept():
    # Generates a random concept
    c = npr.choice([0,1,2], size = (n,), p = [.03,.03,.94])
    print('Generated conjunction: ', humanreadable(c))
    return c

c = concept()


Generated conjunction:   ~x(3) AND ~x(4)


In [276]:
def evaluate(c, z):
    # evaluate a concept c on the given m training vectors arranged in a matrix z.  
    # each row of z is a separate training vector, so z has shape m x n
    m = z.shape[0]
    if len(c) == 0:
        return np.zeros((m,))
    discrepancies = [ y for y in range(m) for x in range(n) if z[y,x] != c[x] and c[x] !=2 ]
    labels = np.ones((m,))
    labels[list(set(discrepancies))] = 0
    return labels

def generateone(m, c):
    # Generate m examples from a distribution D_1. m needs to be a multiple of 5.
    # Assignment: can you describe what this distribution is?
    z = np.tile(npr.choice([0,1], size = (m,5)), int(n/5))
    labels = evaluate(c,z)
    return z, labels
    
m = 100
z, labels = generateone(m, c)
print('Training vectors: \n', z)
print('labels', labels)

Training vectors: 
 [[0 0 0 ... 0 1 1]
 [1 0 1 ... 1 0 1]
 [1 0 0 ... 0 1 0]
 ...
 [1 1 0 ... 0 1 0]
 [1 1 1 ... 1 0 0]
 [1 1 0 ... 0 1 1]]
labels [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 0. 1. 0.]


In [278]:
# Algorithm to infer the conjunction from training vectors z and their labels. 
# This ignores all training vectors whose label is 0. 
# If there is a training vector with label 1, it discards all literals inconsistent with the example.
# If no label is one, estimate concept is [], which evaluates to 0 everywhere
# Verify the output hatc is always consistent with the examples it is trained on.
hatc = []
for i in range(m):
    if labels[i] == 1:
        hatc = z[i,:]
        break
        
for i in range(m):
    if labels[i] == 1:
        discard = np.where(hatc != z[i,:])
        hatc[discard] = 2
        
print('Estimated concept:', humanreadable(hatc))
print('True concept:     ', humanreadable(c))

Estimated concept:  ~x(3) AND ~x(4) AND ~x(8) AND ~x(9) AND ~x(13) AND ~x(14)
True concept:       ~x(3) AND ~x(4)


In [279]:
# If you test it on samples generated by the training distribution:
ztest, truelabels = generateone(10000,c)

estimatedlabels = evaluate(hatc, ztest)

In [280]:
# all is well
gen_acc = np.size(np.where(truelabels == estimatedlabels))/10000
print("Generalization accuracy:", gen_acc)

Generalization accuracy: 1.0


In [281]:
# But if we use a different distribution? 
# You can almost always generate a distribution below that yields high error probability.

def generatetwo(m,c):
    # Can you modify this distribution so as to give a high error probability for the hatc you 
    # obtained in the previous cells?
    z = npr.choice([0,1], size = (m,n), p = [0.9,0.1])    
    z[:,8] = 1
    z[:,9] = 1
    labels = evaluate(c,z)
    return z, labels

ztest, trueoodlabels = generatetwo(10000,c)
estimatedoodlabels = evaluate(hatc, ztest)
ood_acc = np.size(np.where(trueoodlabels == estimatedoodlabels))/10000
print('Out of distribution accuracy:', ood_acc)

Out of distribution accuracy: 0.1919
